In [1]:
_ = '''
transaction = contract.newEvent({
    'currencyPair': 'XTZ-USD',
    'targetDynamics': 1000000,
    'betsCloseTime': int(time.time()) + 3600,
    'measurePeriod': 3600,
    'liquidityPercent': 10000,
}).as_transaction()

transaction.json_payload()#.hash()
'''

### Juster-Maker:
Tool allowing to run events in cycle

In [2]:
from pytezos import pytezos
import time
from pprint import pprint
from utility import repeat_until_succeed
from bulk_sender import BulkSender
from event_emitter import EventCreationEmitter


# pytezos = pytezos.using(shell='https://florencenet-tezos.giganode.io')
pytezos = pytezos.using(shell='https://api.tez.ie/rpc/florencenet')

In [3]:
_ = '''
CONFIG = {
    'events': [],
    'juster_address': 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
}
'''

In [4]:
address = 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
client = pytezos.using(
    key='../2021-05-27-tests-modelling/test-keys/tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ.json'
)
contract = client.contract(address)

In [5]:
import time
time.time()

1626715973.9891067

In [6]:
from datetime import datetime

def date_to_timestamp(date, time_format='%Y-%m-%d %H:%M:%S'):
    """ Converts date to timestamp using by default
        '%Y-%m-%d %H:%M:%S' format """

    return datetime.strptime(date, time_format).timestamp()

date_to_timestamp('2021-07-19 20:00:00')

1626714000.0

### TODO:
- starts_from parameter for events creators so it would be possible to bind events to the time slots

In [7]:
from asyncio import Queue

# TODO: what would happen if queue overflow?
MAX_QUEUE_SIZE = 50

operations_queue = Queue(MAX_QUEUE_SIZE)

events_params = [
    {
        'currency_pair': currency_pair,
        # 'first_at': 
        'target_dynamics': 1_000_000,
        'bets_period': period,
        'measure_period': period,
        'liquidity_percent': 10_000,
        'expiration_fee': 100_000,
        'measure_start_fee': 100_000
    } for currency_pair in ['XTZ-USD', 'BTC-USD']
    for period in [600, 900]
]

In [8]:
# for each event_params EventCreationEmitter is created:
emitters = [
    EventCreationEmitter(
        period=params['bets_period'],
        contract=contract,
        operations_queue=operations_queue,
        event_params=params)
    for params in events_params
]

emitters.append(
    BulkSender(period=60, client=client, operations_queue=operations_queue)
)

In [9]:
for emitter in emitters:
    emitter.run()

created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626716477, 'measurePeriod': 600, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626716777, 'measurePeriod': 900, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'BTC-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626716477, 'measurePeriod': 600, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'BTC-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626716777, 'measurePeriod': 900, 'liquidityPercent': 10000}
making bulk of 4 operations
catched <class 'AttributeError'> in sign: 'dict' object has no attribute 'hash'


### TODO: need to test all this code before running in florencenet

In [1]:
for emitter in emitters:
    emitter.stop()

NameError: name 'emitters' is not defined